In [1]:
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
dataset_inflammation = pd.read_table('AcuteInflammations.data', sep=',')
print("\nDataset Acute Inflammations:")
dataset_inflammation


Dataset Acute Inflammations:


,temperature,occurrence_of_nausea,lumbar_pain,urine_pushing,micturition_pains,burning_of_urethra,decision_1,decision_2
0,35.5,0,1,0,0,0,0,0
1,35.9,0,0,1,1,1,1,0
2,35.9,0,1,0,0,0,0,0
3,36.0,0,0,1,1,1,1,0
4,36.0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
115,41.4,0,1,1,0,1,0,1
116,41.5,0,0,0,0,0,0,0
117,41.5,1,1,0,1,0,0,1
118,41.5,0,1,1,0,1,0,1


In [2]:
dataset_inflammation.columns

Index(['temperature', 'occurrence_of_nausea', 'lumbar_pain', 'urine_pushing',
       'micturition_pains', 'burning_of_urethra', 'decision_1', 'decision_2'],
      dtype='object')

In [3]:
inflammation_values = dataset_inflammation.iloc[:,0:8].values
print("\nAcute Inflammations features:\n")
print(inflammation_values)


Acute Inflammations features:

[[35.5  0.   1.   0.   0.   0.   0.   0. ]
 [35.9  0.   0.   1.   1.   1.   1.   0. ]
 [35.9  0.   1.   0.   0.   0.   0.   0. ]
 [36.   0.   0.   1.   1.   1.   1.   0. ]
 [36.   0.   1.   0.   0.   0.   0.   0. ]
 [36.   0.   1.   0.   0.   0.   0.   0. ]
 [36.2  0.   0.   1.   1.   1.   1.   0. ]
 [36.2  0.   1.   0.   0.   0.   0.   0. ]
 [36.3  0.   0.   1.   1.   1.   1.   0. ]
 [36.6  0.   0.   1.   1.   1.   1.   0. ]
 [36.6  0.   0.   1.   1.   1.   1.   0. ]
 [36.6  0.   1.   0.   0.   0.   0.   0. ]
 [36.6  0.   1.   0.   0.   0.   0.   0. ]
 [36.7  0.   0.   1.   1.   1.   1.   0. ]
 [36.7  0.   1.   0.   0.   0.   0.   0. ]
 [36.7  0.   1.   0.   0.   0.   0.   0. ]
 [36.8  0.   0.   1.   1.   1.   1.   0. ]
 [36.8  0.   0.   1.   1.   1.   1.   0. ]
 [36.9  0.   0.   1.   1.   1.   1.   0. ]
 [36.9  0.   1.   0.   0.   0.   0.   0. ]
 [37.   0.   0.   1.   1.   0.   1.   0. ]
 [37.   0.   0.   1.   1.   0.   1.   0. ]
 [37.   0.   1.   0.  

In [4]:
inflammation_normalizada = dataset_inflammation.values.copy()
normalizador =  StandardScaler()

inflammation_normalizada = normalizador.fit_transform(dataset_inflammation)
dataset_inflammation['temperature'] = inflammation_normalizada[:,0]

print("\nDataset Acute Inflammation normalizada:")
dataset_inflammation


Dataset Acute Inflammation normalizada:


,temperature,occurrence_of_nausea,lumbar_pain,urine_pushing,micturition_pains,burning_of_urethra,decision_1,decision_2
0,-1.779796,0,1,0,0,0,0,0
1,-1.558989,0,0,1,1,1,1,0
2,-1.558989,0,1,0,0,0,0,0
3,-1.503788,0,0,1,1,1,1,0
4,-1.503788,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
115,1.477107,0,1,1,0,1,0,1
116,1.532308,0,0,0,0,0,0,0
117,1.532308,1,1,0,1,0,0,1
118,1.532308,0,1,1,0,1,0,1


In [5]:
inflammation_classes = dataset_inflammation.iloc[:,7].values
print("\nAcute Inflammations classes:\n")
print(inflammation_classes)
print("\nAcute Inflammations classes shape:")
print(inflammation_classes.shape)


Acute Inflammations classes:

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1
 0 1 1 1 1 0 1 1 1]

Acute Inflammations classes shape:
(120,)


In [6]:
kfold_treinamento = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=None)
kfold_ajuste_parametros = model_selection.StratifiedKFold(n_splits=3, shuffle=True, random_state=None)

In [15]:
arvore_decisao = DecisionTreeClassifier()
print("====== Iniciando busca pelos melhores parâmetros do algoritmo Arvore de decisao no dataset Acute Inflammations ....\n")
param_dist = {'max_depth':list(np.arange(1,10)), 'min_samples_split':list(np.arange(1,10)) ,'criterion':['entropy'], 'splitter':['best','random']}
grid_search = GridSearchCV(arvore_decisao, param_grid=param_dist, cv=kfold_ajuste_parametros, scoring='accuracy', refit=False)
grid_search.fit(inflammation_values, inflammation_classes)
decisionTreeBestParams = grid_search.best_params_
print("Decision Tree: %s \n\n" % decisionTreeBestParams)

====== Iniciando busca pelos melhores parâmetros do algoritmo Arvore de decisao no dataset Acute Inflammations ....

Decision Tree: {'criterion': 'entropy', 'max_depth': 1, 'splitter': 'best'} 




In [8]:
knn_vizinhos_proximos = KNeighborsClassifier()
print("====== Iniciando busca pelos melhores parâmetros do algoritmo KNN no dataset Acute Inflammations ....\n")
param_dist = {'n_neighbors':[3,5,7,9], 'metric':['euclidean', 'manhattan'], 'weights':['uniform', 'distance']} #Parâmetros testados
grid_search = GridSearchCV(knn_vizinhos_proximos, param_grid=param_dist, cv=kfold_ajuste_parametros, scoring='accuracy', refit=False)
grid_search.fit(inflammation_values, inflammation_classes)
knnBestParams = grid_search.best_params_
print("KNN: %s \n\n" % knnBestParams)

====== Iniciando busca pelos melhores parâmetros do algoritmo KNN no dataset Acute Inflammations ....

KNN: {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'} 




In [9]:
regressao_logistica = LogisticRegression()
print("====== Iniciando busca pelos melhores parâmetros do algoritmo Regressao logistica no dataset Acute Inflammations ....\n")
param_dist = {'penalty':['l2','l1'], 'max_iter':[200,300,400,500],'solver':['liblinear','sag','saga']} #Parâmetros testados
grid_search = GridSearchCV(regressao_logistica, param_grid=param_dist, cv=kfold_ajuste_parametros, scoring='accuracy', refit=False)
grid_search.fit(inflammation_values, inflammation_classes)
regressao_logistica_param = grid_search.best_params_
print("Regressão logistica: %s \n\n" % regressao_logistica_param)


====== Iniciando busca pelos melhores parâmetros do algoritmo Regressao logistica no dataset Acute Inflammations ....



/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached whi

Regressão logistica: {'max_iter': 300, 'penalty': 'l2', 'solver': 'lbfgs', 'tol': 0.0001} 




/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [10]:
rede_neural = MLPClassifier()
print("====== Iniciando busca pelos melhores parâmetros do algoritmo Redes neurais no dataset Acute Inflammations ....\n")
param_dist = {'hidden_layer_sizes':list(np.arange(10,20)),'activation':['logistic','identity','tanh'], 'max_iter':[100,200,300,400],'alpha':[0.0001,0.001, 0.01, 0.1], 'solver':['sgd','adam']} #Parâmetros testados
grid_search = GridSearchCV(rede_neural, param_grid=param_dist, cv=kfold_ajuste_parametros, scoring='accuracy', refit=False)
grid_search.fit(inflammation_values, inflammation_classes)
redes_neurais_param = grid_search.best_params_
print("Redes neurais: %s \n\n" % redes_neurais_param)


====== Iniciando busca pelos melhores parâmetros do algoritmo Redes neurais no dataset Acute Inflammations ....



/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_pe

/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_pe

/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_pe

/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_pe

...

/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_pe

/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_pe

/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_pe

/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Redes neurais: {'activation': 'logistic', 'alpha': 0.0001, 'hidden_layer_sizes': 5, 'learning_rate': 'constant', 'learning_rate_init': 0.01, 'max_iter': 300, 'solver': 'adam'} 




In [13]:
predicted_classes = dict()
predicted_classes['arvore_decisao'] = np.zeros(inflammation_classes.shape)
predicted_classes['vizinhos_proximos'] = np.zeros(inflammation_classes.shape)
predicted_classes['naive_bayes_gaussian'] = np.zeros(inflammation_classes.shape)
predicted_classes['regressao_logistica'] = np.zeros(inflammation_classes.shape)
predicted_classes['rede_neural'] = np.zeros(inflammation_classes.shape)

arvore_decisao_best_param = DecisionTreeClassifier(**decisionTreeBestParams)
knn_vizinhos_proximos_best_param = KNeighborsClassifier(**knnBestParams)
naive_bayes_gaussian = GaussianNB()
regressao_logistica_best_param = LogisticRegression(**regressao_logistica_param)
rede_neural_best_param = MLPClassifier(**redes_neurais_param)

Iteration 1, loss = 0.71876428
Iteration 2, loss = 0.71491348
Iteration 3, loss = 0.70995496
Iteration 4, loss = 0.70435792
Iteration 5, loss = 0.69851881
Iteration 6, loss = 0.69303417
Iteration 7, loss = 0.68867195
Iteration 8, loss = 0.68572933
Iteration 9, loss = 0.68387657
Iteration 10, loss = 0.68267400
Iteration 11, loss = 0.68183966
Iteration 12, loss = 0.68122744
Iteration 13, loss = 0.68076302
Iteration 14, loss = 0.68040564
Iteration 15, loss = 0.68013020
Iteration 16, loss = 0.67991944
Iteration 17, loss = 0.67976048
Iteration 18, loss = 0.67964314
Iteration 19, loss = 0.67955913
Iteration 20, loss = 0.67950155
Iteration 21, loss = 0.67946464
Iteration 22, loss = 0.67944358
Iteration 23, loss = 0.67943435
Iteration 24, loss = 0.67943362
Iteration 25, loss = 0.67943867
Iteration 26, loss = 0.67944730
Iteration 27, loss = 0.67945778
Iteration 28, loss = 0.67946876
Iteration 29, loss = 0.67947923
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. S

/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Iteration 127, loss = 0.64581174
Iteration 128, loss = 0.64538757
Iteration 129, loss = 0.64495913
Iteration 130, loss = 0.64452637
Iteration 131, loss = 0.64408926
Iteration 132, loss = 0.64364776
Iteration 133, loss = 0.64320185
Iteration 134, loss = 0.64275147
Iteration 135, loss = 0.64229660
Iteration 136, loss = 0.64183721
Iteration 137, loss = 0.64137325
Iteration 138, loss = 0.64090470
Iteration 139, loss = 0.64043151
Iteration 140, loss = 0.63995365
Iteration 141, loss = 0.63947108
Iteration 142, loss = 0.63898378
Iteration 143, loss = 0.63849171
Iteration 144, loss = 0.63799482
Iteration 145, loss = 0.63749309
Iteration 146, loss = 0.63698649
Iteration 147, loss = 0.63647497
Iteration 148, loss = 0.63595850
Iteration 149, loss = 0.63543705
Iteration 150, loss = 0.63491059
Iteration 151, loss = 0.63437908
Iteration 152, loss = 0.63384249
Iteration 153, loss = 0.63330078
Iteration 154, loss = 0.63275392
Iteration 155, loss = 0.63220188
Iteration 156, loss = 0.63164463
Iteration 

/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [14]:

for train, test in kfold_treinamento.split(inflammation_values, inflammation_classes):
    data_train, target_train = inflammation_values[train], inflammation_classes[train]
    data_test, target_test = inflammation_values[test], inflammation_classes[test]

    arvore_decisao = arvore_decisao.fit(data_train, target_train)
    arvore_decisao_predicted = arvore_decisao.predict(data_test)
    predicted_classes['arvore_decisao'][test] = arvore_decisao_predicted

    vizinhos_proximos = vizinhos_proximos.fit(data_train, target_train)
    vizinhos_proximos_predicted = vizinhos_proximos.predict(data_test)
    predicted_classes['vizinhos_proximos'][test] = vizinhos_proximos_predicted

    naive_bayes_gaussian = naive_bayes_gaussian.fit(data_train, target_train)
    naive_bayes_gaussian_predicted = naive_bayes_gaussian.predict(data_test)
    predicted_classes['naive_bayes_gaussian'][test] = naive_bayes_gaussian_predicted

    regressao_logistica = regressao_logistica.fit(data_train, target_train)
    regressao_logistica_predicted = regressao_logistica.predict(data_test)
    predicted_classes['regressao_logistica'][test] = regressao_logistica_predicted

    rede_neural = rede_neural.fit(data_train, target_train)
    rede_neural_predicted = rede_neural.predict(data_test)
    predicted_classes['rede_neural'][test] = rede_neural_predicted

Resultados do classificador arvore_decisao
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        70
           1       1.00      1.00      1.00        50

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120


Matriz de confusão: 
[[70  0]
 [ 0 50]]



Resultados do classificador vizinhos_proximos
              precision    recall  f1-score   support

           0       0.91      0.86      0.88        70
           1       0.81      0.88      0.85        50

    accuracy                           0.87       120
   macro avg       0.86      0.87      0.86       120
weighted avg       0.87      0.87      0.87       120


Matriz de confusão: 
[[60 10]
 [ 6 44]]



Resultados do classificador naive_bayes_gaussian
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        70
           